## Backpropagation Example Implementation

This example has been adapted form the below blog post:

https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/

For the mathematical explaination read my blog post and neural network fundamentals series here:

https://samzee.net/2019/02/20/neural-networks-learning-the-basics-backpropagation/

The standard process for building the neural network is as follows:
1. Initialize Network
2. Forward Propagate
3. Back Propagate Error
4. Train Network

### Reproduce example in blog post

In [43]:
import numpy as np
import random
import io
import pandas as pd
import plotly.express as px
import numpy
import plotly

## Example

In [20]:
def initialze_network(n_input, n_hidden, n_output):
    network = []
    hidden_layer = {'weights': [[np.random.rand(n_input,n_hidden)] , [0]]}
    network.append(hidden_layer)
    output_layer = {'weights': [[np.random.rand(n_hidden,n_output)] , [random.random()]]}
    network.append(output_layer)
    return network
#forward pass
def forward_pass(weights, inputs, bias):
    a = np.dot(weights, inputs) + bias
    return a
def out_gradient(target, output, inputs):
    gradient = (dloss(target, output))*inputs
    return gradient
def bias_gradient(target, output):
    gradient = dloss(target, output)
    return gradient
#one hidden layer back prop
def hid_gradient(target, output, inputs, a, weight):
    k = np.sum((-(target - output))*weight) #the first column of a matrix
    g = a*(1- a)*inputs
    return g*k
def sigmoid(x):
    s = 1/(1 + np.exp(-x))
    return s
#derivative of the loss function delta rule
def dloss(target, output):
    error = -(target - output)
    return error
#loss function
def lossfunction(target,output):
    loss = (1/2)*(target - output)**2
    return loss

In [64]:
def train_network(network, inputs, target, l, n_epoch):
    weights = [neuron['weights'] for neuron in network]
    for epoch in range(n_epoch):
        myloss = []
        h_gradients = []
        for i in range(len(inputs)):
            a_hidden = forward_pass(weights[0][0][0], inputs[i], weights[0][1])
            a_hidden = sigmoid(a_hidden) #apply activation
            outputs = forward_pass(weights[1][0][0].T, a_hidden, weights[1][1])
            #outputs = sigmoid(outputs)
            loss = lossfunction(target[i], outputs)
            t_loss = np.sum(loss)
            myloss.append(t_loss)
            grad = [out_gradient(target[i], outputs, item) for item in a_hidden]
            #print("this is output gradient {} for iteration {}".format(grad, i))
            gradients = np.vstack(grad).T     #output gradients
            bias_grad = bias_gradient(target[i], outputs) 
            w = np.array(weights[1][0]).T
            for item in w:
                grad = hid_gradient(target[i], outputs, inputs[i], a_hidden[0], item)
                h_gradients.append(grad)
                mygrad = np.vstack(h_gradients).T
            weights[0][0] = weights[0][0] - l*mygrad
            weights[1][0] = weights[1][0] - l*gradients
            weights[1][1] = weights[1][1] - l*bias_grad
            final_loss = np.sum(myloss)
            #print('>epoch={}, error={}'.format(n_epoch, final_loss))
            return final_loss

In [82]:
#training samples
inputs = np.array([[2.7810836, 2.550537003], [1.465489372, 2.362125076], [2.396561688, 4.400293529], [1.38807019, 1.850220317],
                  [3.06407232, 3.005305973],[7.627531214, 2.759262235],[5.332441248, 2.088626775],[6.922596716, 1.77106367],
                  [8.675418651, 0.242068655], [7.673756466, 3.508563011]])
target = np.array([[0], [0], [0], [0], [0], [1], [1], [1], [1], [1]])

Dataset from:

https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

An alternative implementation can also be accessed via this link

In [83]:
#initialize network
network = initialze_network(inputs.shape[1], 2, target.shape[1])
weights = [neuron['weights'] for neuron in network]

In [84]:
n_epoch = [x for x in range(1, 20)]
loss = []
for n in n_epoch:
    myloss = train_network(network, inputs, target, 0.5, n)
    loss.append(myloss)

Our neural network works as we see the decrease in the loss function with each epoch

In [85]:
import plotly.graph_objects as go
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=n_epoch[1:], y=loss[1:]))
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)',
    legend_orientation="h",
    autosize=False,
    width=400,
    height=350,
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="epoch",
            font=dict(
                family="Arial Narrow",
                size=18,
                color="#7f7f7f"))),
    yaxis=go.layout.YAxis(
    title=go.layout.yaxis.Title(
        text="loss",
        font=dict(
                family="Arial Narrow",
                size=18,
                color="#7f7f7f"
            )
        )
    )
    
)
fig.update_xaxes(showgrid = True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid = True, gridwidth=1, gridcolor='LightGrey')

fig.show()

In [86]:
plotly.io.orca.config.executable = "C:\\Users\\samantha.vandermerwe\\AppData\\Local\\Programs\\orca\\orca.exe"
fig.write_image("Images/noactivation_scaled.png")